# **Flight Delay Prediction using Scikit-Learn Pipeline**

## **Overview**
This project demonstrates how to build a **machine learning pipeline** using scikit-learn to predict flight delays. The pipeline integrates data preprocessing with model training, ensuring efficient handling of both numerical and categorical data.

---

## **Objectives**
- Preprocess numerical and categorical data using `ColumnTransformer`.
- Automate the machine learning workflow using `Pipeline`.
- Train a **Random Forest Classifier** to predict flight delays.
- Optimize the model using **GridSearchCV** for hyperparameter tuning.

---

## **Data Overview**
- **Dataset**: Contains flight details such as:
  - **Year**, **Month**, **Day**
  - **Airline code**, **Origin airport code**, **Destination airport code**
  - **Departure delay** (target: delayed or not)

- **Target Variable**:  
  - `1` if the flight was delayed  
  - `0` if the flight was on time

---

## **Steps Involved**

### 1. **Data Loading and Exploration**
- Load the flight dataset and inspect its structure and missing values.

### 2. **Feature Engineering**
- **Numerical Features**:
  - `YEAR`, `MONTH`, `DAY`
- **Categorical Features**:
  - `AIRLINE__CODE`, `ORIGIN_AIRPORT_CODE`, `DESTINATION_AIRPORT_CODE`

### 3. **Preprocessing with `ColumnTransformer`**
- **Numerical Data**:
  - Impute missing values with the **mean**.
  - Standardize values using **`StandardScaler`**.
  
- **Categorical Data**:
  - Impute missing values with `'missing'`.
  - Encode using **`OneHotEncoder`**.

### 4. **Pipeline Setup**
- Use a **scikit-learn Pipeline** to link preprocessing and model training.
- Integrate a **Random Forest Classifier** within the pipeline.

### 5. **Model Training and Evaluation**
- Split the data into **train (70%)** and **test (30%)** sets.
- Evaluate the model using a **classification report** with metrics like:
  - **Precision**, **Recall**, **F1-score**

### 6. **Hyperparameter Tuning with GridSearchCV**
- Tune hyperparameters of the Random Forest model:
  - Number of estimators (`n_estimators`)
  - Maximum tree depth (`max_depth`)

### 7. **Model Persistence**
- Save the trained model using **`joblib`** for later use.

---

## **Technologies Used**
- **Python**: Programming language
- **Pandas**: Data manipulation and cleaning
- **Scikit-Learn**: Machine learning, preprocessing, and model evaluation
- **Joblib**: Model persistence
- **Jupyter Notebook**: Interactive development environment

---

## **Expected Output**
- A **trained Random Forest model** to predict flight delays.
- **Performance metrics** (accuracy, precision, recall) from the classification report.
- A **saved model** (`flight_delay_classifier.pkl`) for deployment.

---

## **Conclusion**
This project demonstrates how to create an automated **machine learning workflow** using scikit-learn’s `Pipeline` and `ColumnTransformer`. The streamlined preprocessing ensures consistency during both training and testing. With **hyperparameter tuning**, the model's performance is further optimized, making it reliable for real-world flight delay predictions.


# Upgrade pip and install all required packages

In [ ]:
!pip install --upgrade pip

# Install Snowflake connectors, pandas integration, and essential libraries
!pip install "snowflake-connector-python[pandas]" \
             snowflake-snowpark-python==1.9.0 \
             numpy pandas matplotlib scikit-learn xgboost seaborn \
             python-dateutil tqdm holidays faker

# Ensure Snowpark Python is up-to-date
!pip install --upgrade -q snowflake-snowpark-python==1.9.0

# Fix potential urllib3 version conflicts
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15

# Additional installations for your project
!pip install fosforml==1.1.6
!pip install python-scipy
!pip install basemap


# Importing necessary libraries and settings

In [1]:

# Standard libraries for date and warnings
import datetime
import warnings

# Scientific and Data Manipulation Libraries
import scipy
import pandas as pd
import numpy as np

# Data Visualization Libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb

# Sklearn Modules for Data Preprocessing, Modeling, and Evaluation
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder  # Encoding categorical variables
from sklearn.preprocessing import StandardScaler  # Scaling numerical data
from sklearn.tree import DecisionTreeClassifier  # Decision Tree model
from sklearn.metrics import roc_auc_score, classification_report  # Evaluation metrics

# Configuring display options and warning filters
pd.options.display.max_columns = 50
warnings.filterwarnings("ignore")

# Custom FosforML package for Snowflake session and model registration
from fosforml.model_manager.snowflakesession import get_session
from fosforml import register_model


In [ ]:
# Set Matplotlib's default font family to 'DeJavu Serif' to ensure a consistent font style across plots
plt.rcParams['font.family'] = 'DeJavu Serif'

# Establishing a Snowflake session


In [2]:
my_session = get_session()

# Defining the table name to fetch data from
# table_name = 'FLIGHTS'  # Initial option for table
table_name = 'FLIGHTS_FULL'  # Final table to be used

# Querying the data from the specified Snowflake table
sf_df = my_session.sql("SELECT * FROM {}".format(table_name))

# Converting the Snowflake DataFrame to a pandas DataFrame for local processing
df = sf_df.to_pandas()

df

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE__CODE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT_CODE,DESTINATION_AIRPORT_CODE,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,FLY_DATE,AIRLINE,ORIGIN_AIRPORT,ORIGIN_CITY,ORIGIN_STATE,ORIGIN_COUNTRY,ORIGIN_LATITUDE,ORIGIN_LONGITUDE,DEST_AIRPORT,DEST_CITY,DEST_STATE,DEST_COUNTRY,DEST_LATITUDE,DEST_LONGITUDE
0,2024,6,2,2,MQ,3288,N500MQ,ORD,DSM,1048,1048.0,0.0,10.0,1058.0,78.0,60.0,45.0,299,1143.0,5.0,1206,1148.0,-18.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-06-02,American Eagle Airlines Inc.,Chicago O'Hare International Airport,Chicago,IL,USA,41.97960,-87.90446,Des Moines International Airport,Des Moines,IA,USA,41.53493,-93.66068
1,2024,6,2,2,MQ,3319,N902MQ,LFT,DFW,1048,1040.0,-8.0,8.0,1048.0,86.0,85.0,60.0,351,1148.0,17.0,1214,1205.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-06-02,American Eagle Airlines Inc.,Lafayette Regional Airport,Lafayette,LA,USA,30.20528,-91.98766,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720
2,2024,6,2,2,NK,762,N533NK,ATL,ORD,1048,1101.0,13.0,25.0,1126.0,120.0,132.0,81.0,606,1147.0,26.0,1148,1213.0,25.0,0,0,None,12.0,0.0,13.0,0.0,0.0,2024-06-02,Spirit Air Lines,Hartsfield-Jackson Atlanta International Airport,Atlanta,GA,USA,33.64044,-84.42694,Chicago O'Hare International Airport,Chicago,IL,USA,41.97960,-87.90446
3,2024,6,2,2,AA,2484,N3ENAA,DFW,IAH,1049,1051.0,2.0,20.0,1111.0,77.0,71.0,41.0,224,1152.0,10.0,1206,1202.0,-4.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-06-02,American Airlines Inc.,Dallas/Fort Worth International Airport,Dallas-Fort Worth,TX,USA,32.89595,-97.03720,George Bush Intercontinental Airport,Houston,TX,USA,29.98047,-95.33972
4,2024,6,2,2,B6,842,N623JB,SAV,JFK,1049,1057.0,8.0,11.0,1108.0,131.0,107.0,92.0,718,1240.0,4.0,1300,1244.0,-16.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-06-02,JetBlue Airways,Savannah/Hilton Head International Airport,Savannah,GA,USA,32.12758,-81.20214,John F. Kennedy International Airport (New Yor...,New York,NY,USA,40.63975,-73.77893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819074,2024,4,23,4,UA,829,N417UA,EWR,DEN,1529,1528.0,-1.0,17.0,1545.0,273.0,258.0,234.0,1605,1739.0,7.0,1802,1746.0,-16.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,United Air Lines Inc.,Newark Liberty International Airport,Newark,NJ,USA,40.69250,-74.16866,Denver International Airport,Denver,CO,USA,39.85841,-104.66700
5819075,2024,4,23,4,UA,550,N854UA,PDX,DEN,1529,1539.0,10.0,11.0,1550.0,153.0,139.0,123.0,991,1853.0,5.0,1902,1858.0,-4.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,United Air Lines Inc.,Portland International Airport,Portland,OR,USA,45.58872,-122.59750,Denver International Airport,Denver,CO,USA,39.85841,-104.66700
5819076,2024,4,23,4,UA,1572,N73259,EWR,MIA,1529,1528.0,-1.0,16.0,1544.0,186.0,195.0,165.0,1085,1829.0,14.0,1835,1843.0,8.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,United Air Lines Inc.,Newark Liberty International Airport,Newark,NJ,USA,40.69250,-74.16866,Miami International Airport,Miami,FL,USA,25.79325,-80.29056
5819077,2024,4,23,4,US,765,N762US,DTW,CLT,1529,1532.0,3.0,17.0,1549.0,110.0,107.0,82.0,500,1711.0,8.0,1719,1719.0,0.0,0,0,None,NaN,NaN,NaN,NaN,NaN,2024-04-23,US Airways Inc.,Detroit Metropolitan Airport,Detroit,MI,USA,42.21206,-83.34884,Charlotte Douglas International Airport,Charlotte,NC,USA,35.21401,-80.94313
